# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [1]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('/Users/Chachi/Desktop/MISC/Professional/DataScienceWithPython-Udemy/DataScience/emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('/Users/Chachi/Desktop/MISC/Professional/DataScienceWithPython-Udemy/DataScience/emails/ham', 'ham'))


Let's have a look at that DataFrame:

In [2]:
data.head()

,class,message
/Users/Chachi/Desktop/MISC/Professional/DataScienceWithPython-Udemy/DataScience/emails/spam/00001.7848dde101aa985090474a91ec93fcf0,spam,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr..."
/Users/Chachi/Desktop/MISC/Professional/DataScienceWithPython-Udemy/DataScience/emails/spam/00002.d94f1b97e48ed3b553b3508d116e6a09,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
/Users/Chachi/Desktop/MISC/Professional/DataScienceWithPython-Udemy/DataScience/emails/spam/00003.2ee33bc6eacdb11f38d052c44819ba6c,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
/Users/Chachi/Desktop/MISC/Professional/DataScienceWithPython-Udemy/DataScience/emails/spam/00004.eac8de8d759b7e74154f142194282724,spam,##############################################...
/Users/Chachi/Desktop/MISC/Professional/DataScienceWithPython-Udemy/DataScience/emails/spam/00005.57696a39d7d84318ce497886896bf90d,spam,I thought you might like these:\n\n1) Slim Dow...


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [6]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let's try it out:

In [8]:
print(counts)

  (0, 20407)	1
  (0, 28844)	5
  (0, 44554)	1
  (0, 57486)	1
  (0, 21111)	1
  (0, 54131)	1
  (0, 22319)	1
  (0, 27856)	2
  (0, 36946)	2
  (0, 17466)	2
  (0, 3669)	28
  (0, 53112)	1
  (0, 15912)	1
  (0, 3865)	1
  (0, 1193)	1
  (0, 28855)	2
  (0, 22714)	1
  (0, 3777)	1
  (0, 60830)	1
  (0, 38010)	1
  (0, 0)	1
  (0, 2507)	1
  (0, 878)	1
  (0, 38606)	1
  (0, 3793)	1
  :	:
  (2999, 46468)	2
  (2999, 59145)	1
  (2999, 13966)	1
  (2999, 43537)	2
  (2999, 51993)	2
  (2999, 51125)	1
  (2999, 52000)	1
  (2999, 19003)	1
  (2999, 19930)	1
  (2999, 25894)	1
  (2999, 44095)	3
  (2999, 44115)	1
  (2999, 19929)	1
  (2999, 27117)	1
  (2999, 19932)	1
  (2999, 20051)	1
  (2999, 13397)	2
  (2999, 19778)	1
  (2999, 25157)	1
  (2999, 46316)	1
  (2999, 30662)	2
  (2999, 3103)	1
  (2999, 26274)	1
  (2999, 43488)	1
  (2999, 8396)	1


In [4]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], 
      dtype='|S4')

## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.